In [29]:
import open3d as o3d
import numpy as np
import os
import sys
import copy
sys.path.append('..')
import open3d_tutorial as o3dtut
o3dtut.interactive = not 'CI' in os.environ

In [2]:
demo_data =o3d.data.DemoCropPointCloud()


In [3]:
pcd = o3d.io.read_point_cloud(demo_data.point_cloud_path)
pcd

PointCloud with 196133 points.

In [4]:
down_pcd = pcd.voxel_down_sample(voxel_size=0.05)
down_pcd

PointCloud with 4718 points.

In [5]:
down_pcd.estimate_normals(
    search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn = 30)
)

In [6]:
o3d.visualization.draw_geometries([down_pcd])

In [7]:
plane_model,inliers = down_pcd.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=2000)

In [8]:
inliers_cloud = down_pcd.select_by_index(inliers)
inliers_cloud.paint_uniform_color([1,0,0])

PointCloud with 1008 points.

In [9]:
outlier_cloud = down_pcd.select_by_index(inliers,invert=True)
outlier_cloud.paint_uniform_color([0,1,0])

PointCloud with 3710 points.

In [11]:
o3d.visualization.draw_geometries([inliers_cloud,outlier_cloud])

In [12]:
vol  = o3d.visualization.read_selection_polygon_volume(demo_data.cropped_json_path)

In [15]:
chair = vol.crop_point_cloud(outlier_cloud)

In [16]:
chair.paint_uniform_color([0,0,1])

PointCloud with 626 points.

In [17]:
o3d.visualization.draw_geometries([chair])

In [19]:
print(demo_data.data_root)

C:\Users\Developer/open3d_data


In [23]:
color_img = o3d.io.read_image('./data/color.png')
depth_img = o3d.io.read_image('./data/depth.png')

In [24]:
rgbd_img= o3d.geometry.RGBDImage.create_from_color_and_depth(color_img,depth_img)

In [25]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_img,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
    )
)

In [26]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [27]:
def icp(s,t,tran):
    source_temp = copy.deepcopy(s)
    target_temp = copy.deepcopy(t)
    source_temp.paint_uniform_color([1,0,0])
    target_temp.paint_uniform_color([0,1,0])
    source_temp.transform(tran)
    o3d.visualization.draw_geometries([source_temp,target_temp])

In [40]:
source = o3d.io.read_point_cloud(demo_data.point_cloud_path)
target = o3d.io.read_point_cloud(demo_data.point_cloud_path)
threshold = 0.2
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
icp(source,target,trans_init)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [31]:
print("Inline Alignment")
evaluation =o3d.pipelines.registration.evaluate_registration(source,target,threshold,trans_init)
print(evaluation)

Inline Alignment
RegistrationResult with fitness=6.683730e-02, inlier_rmse=1.185667e-02, and correspondence_set size of 13109
Access transformation to get result.


In [41]:
print("Alignment")
reg2pip= o3d.pipelines.registration.registration_icp(source,target,threshold,trans_init,o3d.pipelines.registration.TransformationEstimationPointToPoint(),o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=4000))
print("Transformation Matrix: ")
print(reg2pip.transformation)
icp(source,target,reg2pip.transformation)

Alignment
Transformation Matrix: 
[[ 9.99766953e-01 -9.45072252e-04 -5.03828504e-04 -2.37788272e-03]
 [ 1.98828533e-04  1.00011674e+00  2.27523632e-04 -1.01727557e-03]
 [-3.34066753e-07 -7.84830676e-04  1.00024932e+00  1.74520140e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
